In [8]:
import numpy as np
import pandas as pd
#import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths

from subprocess import check_output

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import preprocess_input


In [6]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(224, 224))#.convert('L')
    img = img_to_array(img)
    return img

In [14]:
# loading train images
trainPaths = list(paths.list_images('/Users/ghlgh/Desktop/Courework/CS 543/Final Project/noDup/'))
print(len(trainPaths))
train_images = np.array([get_image(img_path) for img_path in trainPaths])
print(train_images.shape)

9081


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ghlgh/Desktop/Courework/CS\\ 543/Final\\ Project/noDup/18fadb00.jpg'

In [29]:
def to_one_hot(labels):
    one_hot_labels = []
    id2class = dict()
    class2id = dict()      ##!!! class(integer) to id(whale class)
    counter = 0
    nw_count = 0
    for id in labels:
        if id == 'new_whale':
            nw_count = nw_count+1
        if id not in id2class.keys():
            one_hot_labels.append(counter)
            id2class[id] = counter
            class2id[counter] = id
            counter += 1
        else:
            one_hot_labels.append(id2class[id])
    one_hot_labels = to_categorical(one_hot_labels, num_classes = 4251)
    print(nw_count)
    return one_hot_labels, class2id, id2class

In [30]:
df = pd.read_csv("C:/Users/jdu12/Desktop/humpback/train.csv")

labels = df['Id']
labels, class2id, id2class = to_one_hot(labels)
#print(labels[0:10])

810


In [7]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation


In [8]:
# define data generator
#use of an image generator for preprocessing and data augmentation
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

In [9]:
# acquire the model
model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, classes=4251)

In [10]:
batch_size = 32
epochs = 180

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = 'C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}.hdf5"
best_save = ModelCheckpoint(best_dir, monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


model.fit_generator(image_gen.flow(x_train, labels, batch_size=batch_size),
          steps_per_epoch=  x_train.shape[0]//batch_size,
          epochs=epochs, verbose=1, callbacks=[routine_save, best_save])
          #class_weight=class_weight_dic)



Epoch 1/180
307/307 [==============================] - 142s 462ms/step - loss: 8.4272 - acc: 0.0603
Epoch 2/180
307/307 [==============================] - 126s 411ms/step - loss: 7.8322 - acc: 0.0717
Epoch 3/180
307/307 [==============================] - 125s 408ms/step - loss: 7.3146 - acc: 0.0747
Epoch 4/180
307/307 [==============================] - 125s 407ms/step - loss: 6.9169 - acc: 0.0732
Epoch 5/180
307/307 [==============================] - 122s 397ms/step - loss: 6.5652 - acc: 0.0694
Epoch 6/180
307/307 [==============================] - 118s 385ms/step - loss: 6.2235 - acc: 0.0670
Epoch 7/180
307/307 [==============================] - 120s 389ms/step - loss: 5.9016 - acc: 0.0657
Epoch 8/180
307/307 [==============================] - 122s 398ms/step - loss: 5.5103 - acc: 0.0715
Epoch 9/180
307/307 [==============================] - 122s 397ms/step - loss: 5.1645 - acc: 0.0824
Epoch 10/180
307/307 [==============================] - 124s 403ms/step - loss: 4.7750 - acc: 0.0989

307/307 [==============================] - 105s 342ms/step - loss: 0.1017 - acc: 0.9689
Epoch 164/180
307/307 [==============================] - 105s 342ms/step - loss: 0.0968 - acc: 0.9703
Epoch 165/180
307/307 [==============================] - 105s 342ms/step - loss: 0.1016 - acc: 0.9691
Epoch 166/180
307/307 [==============================] - 105s 343ms/step - loss: 0.0997 - acc: 0.9688
Epoch 167/180
307/307 [==============================] - 105s 342ms/step - loss: 0.0982 - acc: 0.9686
Epoch 168/180
307/307 [==============================] - 105s 342ms/step - loss: 0.1005 - acc: 0.9685
Epoch 169/180
307/307 [==============================] - 112s 366ms/step - loss: 0.0978 - acc: 0.9675
Epoch 170/180
307/307 [==============================] - 120s 392ms/step - loss: 0.1013 - acc: 0.9674
Epoch 171/180
307/307 [==============================] - 117s 382ms/step - loss: 0.0950 - acc: 0.9692
Epoch 172/180
307/307 [==============================] - 118s 383ms/step - loss: 0.0975 - acc: 0

In [8]:
# loading test images
testPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/test/'))
print(len(testPaths))
test_images = np.array([get_image(img_path) for img_path in testPaths])/255
print(test_images.shape)

15610
(15610, 224, 224, 3)


In [58]:
# loading trained model
model = load_model('C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/' + 'routine-150-0.97.hdf5')

In [59]:
# predict!
pred = model.predict(test_images)

In [60]:
# testing and generating submission file
import warnings
from os.path import split

with open("sample_submission.csv","w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))     

In [52]:
top_5 = np.argsort(pred)[:,-1:-6:-1]
print(top_5.shape)
a = np.asarray([[1, 2, 3, 4, 5, 6, 7, 8],[1, 2, 3, 4, 5, 6, 7, 8]])
print(a[:,-1:-6:-1])


[[8 7 6 5 4]
 [8 7 6 5 4]]
